In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/microsoft/llava-rad.git
%cd llava-rad

fatal: destination path 'llava-rad' already exists and is not an empty directory.
/content/llava-rad


In [ ]:
!pip install --upgrade pip
!pip install -e .


Obtaining file:///content/llava-rad
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llava (pyproject.toml) ... done
  Created wheel for llava: filename=llava-1.1.1-0.editable-py3-none-any.whl size=10918 sha256=b9e85ea15c85e62112b73d1cc4e9358323140612833e7228fd4e7780353fa2f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-wozxrl5j/wheels/56/5b/c8/967271f67eb2bc950867cb2037eb428d29b4ef1c06b88928bd
Successfully built llava
  Attempting uninstall: llava
    Found existing installation: llava 1.1.1
    Uninstalling llava-1.1.1:
      Successfully uninstalled llava-1.1.1


In [ ]:
import requests
import torch
from PIL import Image
from io import BytesIO

from llava.constants import IMAGE_TOKEN_INDEX
from llava.conversation import conv_templates
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, KeywordsStoppingCriteria


def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

# Model
disable_torch_init()

model_path = "microsoft/llava-rad"
model_base = "lmsys/vicuna-7b-v1.5"
model_name = "llavarad"
conv_mode = "v1"

tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)


# Large left pneumothorax is present with apical pneumothorax component
#  measuring approximately 3.4 cm in craniocaudal dimension, and a basilar
#  component overlying the left hemidiaphragm, with visceral pleural line just
#  below the left seventh posterior rib.  Cardiomediastinal contours are normal.
#  The lungs are clear.  No pleural effusion.


[2025-05-12 00:58:36,699] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

Merging LoRA weights...
Model is loaded...
Loading vision tower from HF Hub


biomedclipcxr_518_checkpoint.pt:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

In [ ]:
# Prepare query
input_list = [("/content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic28602.jpg", "Can you understand this chest X Ray?"),
 ("/content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic28602.jpg", """Can you understand this chest X Ray (with additional context)?:

Closest Report Findings:
                                 FINAL REPORT
 EXAMINATION:  CHEST (PORTABLE AP)

 INDICATION:  History: ___F with fall from standing. Reports headache, neck
 pain, Tspine TTP, right shoulder TTP  // eval for ICH, spinal fracture,
 shoulder fracture/fracture

 COMPARISON:  CT T-spine from earlier the same day and chest radiographs ___

 FINDINGS:

 AP semi upright view of the chest provided.

 There is no focal consolidation, effusion, or pneumothorax.  Bibasilar
 atelectasis is similar to prior.  Mild cardiomegaly and large hiatal hernia
 are similar to prior. Imaged osseous structures are intact.  No free air below
 the right hemidiaphragm is seen.

 IMPRESSION:

 No acute intrathoracic process.""")]

for image_file, query in input_list:
  # image_file = "/content/test_img_3.jpg" # CXR w pneumothorax from Open-I
  # query = "<image>Can you undertstand context?"

  conv = conv_templates[conv_mode].copy()
  conv.append_message(conv.roles[0], query)
  conv.append_message(conv.roles[1], None)
  prompt = conv.get_prompt()
  print(prompt)
  image = load_image(image_file)
  image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0].half().unsqueeze(0).cuda()

  input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

  stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)

  with torch.inference_mode():
      output_ids = model.generate(
          input_ids,
          images=image_tensor,
          do_sample=False,
          temperature=0.0,
          max_new_tokens=1024,
          use_cache=True)

  outputs = tokenizer.batch_decode(output_ids[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
  outputs = outputs.strip()
  print(outputs)
  print()
  print("*" * 100)
  print()

A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Can you understand this chest X Ray? ASSISTANT:
As an AI language model, I can analyze and interpret the text within the image of the chest X-ray you provided. However, I do not have the ability to visualize the actual X-ray. Please provide me with the text within the image so I can assist you further.

****************************************************************************************************

A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Can you understand this chest X Ray (with additional context)?:

Closest Report Findings:
                                 FINAL REPORT
 EXAMINATION:  CHEST (PORTABLE AP)

 INDICATION:  History: ___F with fall from standing. Reports headache, neck
 pain, Tspine TTP, righ

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/CS685_Project/VQA_RAD_Chest_Data_Results_Both.csv")
print(df.head())

   q_id                                         image_path  \
0     3  C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...   
1     5  C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...   
2    11  C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...   
3    19  C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...   
4    26  C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...   

                                           query  \
0  What type of imaging does this not represent?   
1            What is not pictured in this image?   
2  Which organ system is abnormal in this image?   
3                   How is the patient oriented?   
4                    What type of image is this?   

                                     graphrag_output  \
0  The type of imaging that is not represented in...   
1  The image does not depict any evidence of pneu...   
2  The organ system that is abnormal in the provi...   
3  The patient is oriented as there is no mention...   
4  The type of image described in 

In [ ]:
df.head(1)

,q_id,image_path,query,graphrag_output,top_k_contents
0,3,C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...,What type of imaging does this not represent?,The type of imaging that is not represented in...,FINAL REPORT\...


In [ ]:
# sample path from my drive: /content/drive/MyDrive/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic100132.jpg

df_test = df.head(5)
df_test

,q_id,image_path,query,graphrag_output,top_k_contents
0,3,C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...,What type of imaging does this not represent?,The type of imaging that is not represented in...,FINAL REPORT\...
1,5,C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...,What is not pictured in this image?,The image does not depict any evidence of pneu...,FINAL REPORT\...
2,11,C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...,Which organ system is abnormal in this image?,The organ system that is abnormal in the provi...,FINAL REPORT\...
3,19,C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...,How is the patient oriented?,The patient is oriented as there is no mention...,FINAL REPORT\...
4,26,C:\Users\arghy\Downloads\VQA-RAD-Chest\VQA_RAD...,What type of image is this?,The type of image described in the context is ...,FINAL REPORT\...


In [ ]:
import os
from pathlib import Path

for i in range(len(df_test)):
  query = df_test["query"][i]
  filename = df_test["image_path"][i].replace("\\", "/").split("/")[-1]
  img_path = "/content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/" + filename

  print(query, img_path)


What type of imaging does this not represent? /content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic28602.jpg
What is not pictured in this image? /content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic28602.jpg
Which organ system is abnormal in this image? /content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic42202.jpg
How is the patient oriented? /content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic29265.jpg
What type of image is this? /content/drive/MyDrive/CS685_Project/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder/synpic42202.jpg


In [ ]:
from __future__ import annotations
import pathlib
from typing import Callable, List
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
import requests
from io import BytesIO


# ---------- tiny helper to fetch local paths **or** URLs --------------------
def _open_image(path_or_url: str | pathlib.Path) -> Image.Image:
    p = str(path_or_url)
    if p.startswith(("http://", "https://")):
        resp = requests.get(p, timeout=15)
        resp.raise_for_status()
        return Image.open(BytesIO(resp.content)).convert("RGB")
    return Image.open(path_or_url).convert("RGB")


# ---------------------------------------------------------------------------
class CSVModelRunner:
    """
    Run *any* model on a dataframe and append its answer.

    Parameters
    ----------
    model_fn    : Callable[[str, str], str]
                  Your function that takes (image_path, query_text) and
                  returns a string answer.
    input_csv   : CSV with at least two columns *or* anything your
                  image_fn/query_fn can handle.
    output_csv  : Where to write the augmented CSV.
    image_fn    : row -> str  (optional transformation)
    query_fn    : row -> str  (optional transformation)
    image_col   : fallback column for image path (default 'image_path')
    query_col   : fallback column for text      (default 'query_text')
    batch_size  : 1 = no batching; if your model_fn accepts lists you
                  can vectorise by changing this class slightly.
    output_col  : name of the new column (default 'model_output')
    """

    def __init__(
        self,
        model_fn: Callable[[str, str], str],
        input_csv: str | pathlib.Path,
        output_csv: str | pathlib.Path,
        *,
        image_fn=None,
        query_fn=None,
        image_col: str = "image_path",
        query_col: str = "query_text",
        output_col: str = "model_output",
        batch_size: int = 1,
    ):
        self.model_fn = model_fn
        self.df = pd.read_csv(input_csv)
        self.output_csv = pathlib.Path(output_csv)
        self.output_col = output_col
        self.batch_size = max(1, batch_size)

        # fall‑back to columns if no transformation given
        if image_fn is None:
            if image_col not in self.df.columns:
                raise ValueError(f"Missing column '{image_col}'")
            image_fn = lambda row: getattr(row, image_col)   # noqa: E731
        if query_fn is None:
            if query_col not in self.df.columns:
                raise ValueError(f"Missing column '{query_col}'")
            query_fn = lambda row: getattr(row, query_col)   # noqa: E731

        self.image_fn = image_fn
        self.query_fn = query_fn

    # --------------------------------------------------------------------
    # def run(self):
    #     answers: List[str] = []
    #     iterator = tqdm(self.df.itertuples(index=False), total=len(self.df))

    #     # single‑row loop (simple & memory‑friendly)
    #     for row in iterator:
    #         img_path = str(self.image_fn(row))
    #         query = self.query_fn(row)

    #         # debug
    #         print(query, " | ", img_path)

    #         answer = self.model_fn(img_path, query)
    #         answers.append(answer)

    #     self.df[self.output_col] = answers
    #     self.output_csv.parent.mkdir(parents=True, exist_ok=True)
    #     self.df.to_csv(self.output_csv, index=False)
    #     print(f"✅  Saved ➜ {self.output_csv.resolve()}")


    def run(self, save_every: int = 1, show_progress: bool = True):
        """
        Parameters
        ----------
        save_every : int
            Flush results to disk after this many rows (last chunk is flushed
            at the end even if < save_every).
        """
        buf_rows, buf_answers = [], []
        first_write = not self.output_csv.exists()

        iterator = (
            tqdm(self.df.itertuples(index=False), total=len(self.df))
            if show_progress else self.df.itertuples(index=False)
        )

        for idx, row in enumerate(iterator):
            img_path = str(self.image_fn(row))
            query    = self.query_fn(row)
            answer   = self.model_fn(img_path, query)

            # keep original row + new answer in a dict
            out_row = row._asdict()
            out_row[self.output_col] = answer
            buf_rows.append(out_row)

            # flush if buffer full
            if len(buf_rows) >= save_every:
                self._flush(buf_rows, first_write)
                first_write = False   # header written once
                buf_rows.clear()

        # flush remainder
        if buf_rows:
            self._flush(buf_rows, first_write)

        print(f"✅  Finished – predictions saved incrementally to "
              f"{self.output_csv.resolve()}")

    # ------------------------------------------------------------------
    def _flush(self, rows_buffer, first_write: bool):
        """Write a list[dict] to CSV (append mode)."""
        pd.DataFrame(rows_buffer).to_csv(
            self.output_csv,
            mode="w" if first_write else "a",
            header=first_write,
            index=False,
        )



def llava_rad_fn(img_path: str, query: str) -> str:
    conv = conv_templates[conv_mode].copy()
    conv.append_message(conv.roles[0], query)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    print(prompt)
    image = load_image(image_file)
    # image = _open_image(img_path)
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0].half().unsqueeze(0).cuda()

    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

    stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=False,
            temperature=0.0,
            max_new_tokens=1024,
            use_cache=True)

    outputs = tokenizer.batch_decode(output_ids[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
    outputs = outputs.strip()
    print(outputs)
    print()
    print("*" * 100)
    print()

    return outputs



In [ ]:
# # 2️⃣  define the simple path fixer for VQA‑RAD
# from pathlib import Path

# RAD_ROOT = Path("/content/drive/MyDrive/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder")

# def image_transform(row):
#     filename = Path(row.image_path.replace("\\", "/")).name
#     return RAD_ROOT / filename


# runner = CSVModelRunner(
#     model_fn     = llava_rad_fn,
#     input_csv    = "/content/VQA_RAD_Chest_Data_Results_Both.csv",
#     output_csv   = "/content/llava_predictions.csv",
#     image_fn     = image_transform,   # use our fixer
#     query_fn     = lambda r: r.query, # untouched
#     batch_size   = 1                  # single‑row inference
# )
# runner.run()


In [ ]:
import gc, torch, re

# --- helper that works on every PyTorch version ----------------------------
def _is_oom_error(exc: BaseException) -> bool:
    """Return True if *exc* looks like a CUDA‑OOM."""
    if isinstance(exc, getattr(torch.cuda, "OutOfMemoryError", RuntimeError)):
        return True
    return isinstance(exc, RuntimeError) and bool(
        re.search(r"CUDA.*out of memory", str(exc), re.I)
    )

# --- drop‑in safe‑inference wrapper ----------------------------------------
def safe_inference(img_path: str, query: str) -> str:
    """
    Run your model once and aggressively free VRAM afterwards.
    Replace the single line with *your* inference call.
    """
    try:
        with torch.no_grad():                     # no grad buffers
            answer = llava_rad_fn(img_path, query)   # <-- your function
        return answer
    except BaseException as e:
        if _is_oom_error(e):
            print("⚠️  Caught CUDA OOM.  Flushing cache and re‑raising.")
            torch.cuda.empty_cache()              # flush allocator cache
        raise                                      # propagate
    finally:
        # Remove stray GPU tensors your code may have created
        for obj_name in ("image_tensor", "input_ids", "output_ids"):
            if obj_name in globals():
                del globals()[obj_name]
        gc.collect()                               # free Python refs
        torch.cuda.empty_cache()                   # free cached blocks :contentReference[oaicite:2]{index=2}
        torch.cuda.ipc_collect()                   # free IPC-handled blocks :contentReference[oaicite:3]{index=3}


In [ ]:
from pathlib import Path

RAD_ROOT = Path("/content/drive/MyDrive/VQA-RAD-Chest/VQA_RAD_Chest_Image_Folder")

def image_transform(row):
    filename = Path(row.image_path.replace("\\", "/")).name
    return RAD_ROOT / filename

runner = CSVModelRunner(
    model_fn  = safe_inference,
    input_csv = "/content/drive/MyDrive/CS685_Project/VQA_RAD_Chest_Data_Results_Both.csv",
    output_csv= "/content/drive/MyDrive/CS685_Project/llava_preds_vqa_rad_only_rag.csv",
    image_fn  = image_transform,
    query_fn  = lambda r: r.query + "\n" + r.top_k_contents,
)
runner.run()


  0%|          | 0/283 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
 IMPRESSION:  Partially loculated right pleural effusion.  Interstitial
 pulmonary edema.  Recommend correlation with chest CT from today.
 || 
                                 FINAL REPORT
 INDICATION:  ___ year old woman with pancreatic cancer, pneumonia, pleural
 effusion likely malignant  // f/u effusion, infiltrates
 
 TECHNIQUE:  Portable
 
 COMPARISON:  ___
 
 FINDINGS: 
 
 New right-sided PICC line with the tip in the low SVC.
 
 Right upper lobe and parahilar opacity has decreased in extent with residual
 masslike area of opacification surrounding the right hilum.  Asymmetric
 interstitial edema has also decreased.  Moderate right and small left pleural
 effusion with bibasal atelectasis slightly increased.  No pneumothorax.
 
 IMPRESSION: 
 
 Right-sided PICC with the tip in the low SVC.
 
 Right upper lobe and juxta hilar opacity have slightly improved.  Persistent
 pleural effusions.
 || 
                                 FI